In [1]:
import random
import shutil
import glob
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.metrics import classification_report

# -----------------------------------
# Kaggle Setup & Data Download
# -----------------------------------
# Make sure you have kaggle.json, which contains {"username":"YOUR_USERNAME","key":"YOUR_KEY"}.
# If you're in a notebook environment (e.g., Colab), upload your kaggle.json or set the environment variables accordingly.

# If running in Google Colab, for example:
# from google.colab import files
# files.upload() # upload kaggle.json

# !mkdir -p ~/.kaggle
# !cp kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json

# Install Kaggle API
!pip install kaggle --quiet

# -----------medical-mnist--------------
# !kaggle datasets download -d andrewmvd/medical-mnist
# !unzip -q medical-mnist.zip -d medical_mnist_data

# --------------natural-images------------
!kaggle datasets download -d prasunroy/natural-images
!unzip -q natural-images.zip -d natural_images_data

Dataset URL: https://www.kaggle.com/datasets/prasunroy/natural-images
License(s): CC-BY-NC-SA-4.0
100% 342M/342M [00:16<00:00, 24.0MB/s]
100% 342M/342M [00:16<00:00, 22.1MB/s]


In [2]:
import os

# -----------------------------------
# Parameters and Paths
# -----------------------------------
# data_dir = "medical_mnist_data/"
# split_base = "medical_mnist_split"


data_dir = "natural_images_data/natural_images"
split_base = "natural_images_split"


img_height, img_width = 224, 224
batch_size = 32
train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15
seed = 42

# -----------------------------------
# Prepare Dataset Splits
# -----------------------------------
root_dir = data_dir
classes = [d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))]


if os.path.exists(split_base):
    shutil.rmtree(split_base)
os.makedirs(split_base, exist_ok=True)

train_dir = os.path.join(split_base, "train")
val_dir = os.path.join(split_base, "val")
test_dir = os.path.join(split_base, "test")

for c in classes:
    os.makedirs(os.path.join(train_dir, c), exist_ok=True)
    os.makedirs(os.path.join(val_dir, c), exist_ok=True)
    os.makedirs(os.path.join(test_dir, c), exist_ok=True)

for c in classes:
    class_path = os.path.join(root_dir, c)
    images = glob.glob(os.path.join(class_path, "*.*"))  # picks up all files
    random.seed(seed)
    random.shuffle(images)
    total = len(images)
    train_count = int(train_ratio * total)
    val_count = int(val_ratio * total)
    test_count = total - train_count - val_count

    train_images = images[:train_count]
    val_images = images[train_count:train_count + val_count]
    test_images = images[train_count + val_count:]

    for img in train_images:
        shutil.copy(img, os.path.join(train_dir, c))
    for img in val_images:
        shutil.copy(img, os.path.join(val_dir, c))
    for img in test_images:
        shutil.copy(img, os.path.join(test_dir, c))

# -----------------------------------
# Create Datasets
# -----------------------------------
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    seed=seed
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    val_dir,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    seed=seed
)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    shuffle=False
)

class_names = train_ds.class_names
num_classes = len(class_names)

AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.prefetch(buffer_size=AUTOTUNE)


Found 4826 files belonging to 8 classes.
Found 1032 files belonging to 8 classes.
Found 1041 files belonging to 8 classes.


In [3]:
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

def VGG16_Modified(input_shape=(224, 224, 3), num_classes=10):
    model = models.Sequential()

    # Block 1
    model.add(layers.Conv2D(64, (3, 3), padding='same',
                            kernel_regularizer=regularizers.l2(1e-4),
                            input_shape=input_shape))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.Conv2D(64, (3, 3), padding='same',
                            kernel_regularizer=regularizers.l2(1e-4)))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling2D((2, 2), strides=(2, 2)))

    # Block 2
    model.add(layers.Conv2D(128, (3, 3), padding='same',
                            kernel_regularizer=regularizers.l2(1e-4)))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.Conv2D(128, (3, 3), padding='same',
                            kernel_regularizer=regularizers.l2(1e-4)))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling2D((2, 2), strides=(2, 2)))

    # Block 3
    model.add(layers.Conv2D(256, (3, 3), padding='same',
                            kernel_regularizer=regularizers.l2(1e-4)))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.Conv2D(256, (3, 3), padding='same',
                            kernel_regularizer=regularizers.l2(1e-4)))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.Conv2D(256, (3, 3), padding='same',
                            kernel_regularizer=regularizers.l2(1e-4)))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.MaxPooling2D((2, 2), strides=(2, 2)))

    # # Block 4
    # model.add(layers.Conv2D(512, (3, 3), padding='same',
    #                         kernel_regularizer=regularizers.l2(1e-4)))
    # model.add(layers.BatchNormalization())
    # model.add(layers.Activation('relu'))
    # model.add(layers.Conv2D(512, (3, 3), padding='same',
    #                         kernel_regularizer=regularizers.l2(1e-4)))
    # model.add(layers.BatchNormalization())
    # model.add(layers.Activation('relu'))
    # model.add(layers.Conv2D(512, (3, 3), padding='same',
    #                         kernel_regularizer=regularizers.l2(1e-4)))
    # model.add(layers.BatchNormalization())
    # model.add(layers.Activation('relu'))
    # model.add(layers.MaxPooling2D((2, 2), strides=(2, 2)))

    # # Block 5
    # model.add(layers.Conv2D(512, (3, 3), padding='same',
    #                         kernel_regularizer=regularizers.l2(1e-4)))
    # model.add(layers.BatchNormalization())
    # model.add(layers.Activation('relu'))
    # model.add(layers.Conv2D(512, (3, 3), padding='same',
    #                         kernel_regularizer=regularizers.l2(1e-4)))
    # model.add(layers.BatchNormalization())
    # model.add(layers.Activation('relu'))
    # model.add(layers.Conv2D(512, (3, 3), padding='same',
    #                         kernel_regularizer=regularizers.l2(1e-4)))
    # model.add(layers.BatchNormalization())
    # model.add(layers.Activation('relu'))
    # model.add(layers.MaxPooling2D((2, 2), strides=(2, 2)))

    # Flatten and Fully Connected Layers
    model.add(layers.Flatten())

    # Reduced Dense Layers
    model.add(layers.Dense(1024, kernel_regularizer=regularizers.l2(1e-4)))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.Dropout(0.5))

    model.add(layers.Dense(512, kernel_regularizer=regularizers.l2(1e-4)))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation('relu'))
    model.add(layers.Dropout(0.5))

    # Output Layer
    model.add(layers.Dense(num_classes, activation='softmax'))

    return model

# Instantiate the Modified VGG16 Model
input_shape = (224, 224, 3)
num_classes = 10  # Adjust based on your dataset
model = VGG16_Modified(input_shape, num_classes)

# Compile the Model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='sparse_categorical_crossentropy',  # Ensure compatibility with labels
    metrics=['accuracy']
)

# Display Model Summary
model.summary()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 224, 224, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 224, 224, 64)        │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation (Activation)              │ (None, 224, 224, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 224, 224, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 224, 224, 64)        │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_1 (Activation)            │ (None, 224, 224, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 112, 112, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 112, 112, 128)       │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 112, 112, 128)       │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_2 (Activation)            │ (None, 112, 112, 128)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 112, 112, 128)       │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 112, 112, 128)       │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_3 (Activation)            │ (None, 112, 112, 128)       │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 56, 56, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 56, 56, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_4                │ (None, 56, 56, 256)         │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_4 (Activation)            │ (None, 56, 56, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_5                │ (None, 56, 56, 256)         │           1,0

 Total params: 207,798,090 (792.69 MB)

 Trainable params: 207,792,714 (792.67 MB)

 Non-trainable params: 5,376 (21.00 KB)

In [ ]:
# -----------------------------------
# Train the Model
# -----------------------------------
epochs = 20
history = model.fit(
    train_ds,
    epochs=epochs,
    validation_data=val_ds
)


Epoch 1/20
151/151 ━━━━━━━━━━━━━━━━━━━━ 142s 661ms/step - accuracy: 0.7299 - loss: 1.2187 - val_accuracy: 0.5281 - val_loss: 1.9195
Epoch 2/20
151/151 ━━━━━━━━━━━━━━━━━━━━ 75s 433ms/step - accuracy: 0.9193 - loss: 0.6331 - val_accuracy: 0.8081 - val_loss: 0.9112
Epoch 3/20
151/151 ━━━━━━━━━━━━━━━━━━━━ 83s 440ms/step - accuracy: 0.9440 - loss: 0.5538 - val_accuracy: 0.9089 - val_loss: 0.6639
Epoch 4/20
151/151 ━━━━━━━━━━━━━━━━━━━━ 82s 438ms/step - accuracy: 0.9651 - loss: 0.4904 - val_accuracy: 0.9196 - val_loss: 0.6123
Epoch 5/20
151/151 ━━━━━━━━━━━━━━━━━━━━ 80s 425ms/step - accuracy: 0.9846 - loss: 0.4521 - val_accuracy: 0.9157 - val_loss: 0.6071
Epoch 6/20
151/151 ━━━━━━━━━━━━━━━━━━━━ 83s 434ms/step - accuracy: 0.9891 - loss: 0.4328 - val_accuracy: 0.9196 - val_loss: 0.6253
Epoch 7/20
151/151 ━━━━━━━━━━━━━━━━━━━━ 83s 440ms/step - accuracy: 0.9868 - loss: 0.4265 - val_accuracy: 0.9331 - val_loss: 0.6272
Epoch 8/20
151/151 ━━━━━━━━━━━━━━━━━━━━ 81s 432ms/step - accuracy: 0.9918 - loss: 

In [ ]:
# -----------------------------------
# Evaluate on Test Set
# -----------------------------------
test_loss, test_acc = model.evaluate(test_ds)
print("Test accuracy:", test_acc)

# -----------------------------------
# Classification Report
# -----------------------------------
y_true = []
y_pred = []

for images, labels in test_ds:
    preds = model.predict(images, verbose=0)
    preds = np.argmax(preds, axis=1)
    y_pred.extend(preds.tolist())
    y_true.extend(labels.numpy().tolist())

print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=class_names, zero_division=0))